In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# (issue) with procedures table

* If ex. 4333 is in both dictionarys with procedure and diagnose label this will either be repeated or overwritten
* 331 is an example of a code repeated in procedure dictionary with different labels, so again wil both be repreated or overwritten

In [ ]:
df_diag  = pd.read_csv('mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz')

In [ ]:
df_diag.head()

# label the proedures with help of ICD9_CODES from bioportal

some of the codes in procedure table contain diseases codes as from the website

https://www.findacode.com/search/search.php?str=3601

we can find the codes and se that exp. 4441 is both in procedures and in diseases, but 360.1 is only in diseases which we have in the procedures table

also 374 is in different catgeory both in procedure and in disease depending on which volume of the icd9 is used

407 is a HCPCS and not a ICD9 code based on this website
https://www.findacode.com/search/search.php?str=407
and therefore not found in the icd9 codes from bio portal, but accurs in the procedure table

In [ ]:
ICD9_df = pd.read_csv('ICD9_CODES.csv')
ICD9_df = ICD9_df[['0', 'LABEL']].rename(columns={'0':'ICD9_CODE'})

ICD9_diag = ICD9_df[ICD9_df['LABEL'] != 'PROCEDURES']
ICD9_proc = ICD9_df[ICD9_df['LABEL'] == 'PROCEDURES']

ICD9_diag['LABEL'].unique()

In [ ]:
df_diag = pd.merge(df_diag, ICD9_diag, on='ICD9_CODE', how='left')
df_diag = df_diag[~df_diag['LABEL'].isnull()]

df_diag

In [ ]:
#df_proc.to_csv('mimic-iii-clinical-database-1.4/PROC_LABELD.csv', index=False)
df_diag.to_csv('mimic-iii-clinical-database-1.4/DIAG_LABELD.csv', index=False)

## Take care of Errors

1. Database where error are given, from the matched
2. Database where the errors are removed

In [ ]:
#df_proc = pd.read_csv('mimic-iii-clinical-database-1.4/PROC_LABELD.csv')
df_diag = pd.read_csv('mimic-iii-clinical-database-1.4/DIAG_LABELD.csv')

In [ ]:
df_diag

In [ ]:
df_diag = df_diag[['HADM_ID', 'ICD9_CODE', 'LABEL']]

df_diag.head()

In [ ]:
df_dis = df_diag[df_diag['LABEL'] == 'DISEASES AND INJURIES'].rename(columns={'ICD9_CODE':'DISEASES AND INJURIES'})
df_dis = df_dis.drop('LABEL', axis=1)
df_dis.head()

In [ ]:
df_symp = df_diag[df_diag['LABEL'] == 'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS'].rename(columns={'ICD9_CODE':'SYMPTOMS, SIGNS, AND ILL-DEFINED CONDITIONS'})
df_symp = df_symp.drop('LABEL', axis=1)
df_symp.head()

In [ ]:
df_fact = df_diag[df_diag['LABEL'] == 'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES'].rename(columns={'ICD9_CODE':'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES'})
df_fact = df_fact.drop('LABEL', axis=1)
df_fact.head()

In [ ]:
df_ext = df_diag[df_diag['LABEL'] == 'SUPPLEMENTARY CLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING'].rename(columns={'ICD9_CODE':'SUPPLEMENTARY CLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING'})
df_ext = df_ext.drop('LABEL', axis=1)
df_ext.head()

In [ ]:
df = pd.merge(df_dis, df_symp, on='HADM_ID', how='inner')
df = pd.merge(df, df_fact, on='HADM_ID', how='inner')
df = pd.merge(df, df_ext, on='HADM_ID', how='inner')

df

In [ ]:
df.to_csv('mimic-iii-clinical-database-1.4/DIAG_ROWS.csv', index=False)

In [ ]:
## Issue how to combine mutiple andhow to handle duplicated values

df = df[df['HADM_ID'] == 199993]

## Add code type as columns to rows

In [ ]:
gb = df.groupby('HADM_ID')

df = pd.DataFrame(dtype='str')

for name, group in tqdm(gb):
    
    df_new = pd.DataFrame([np.append(group['ICD9_CODE'], name)], columns=np.append(group['LABEL'].values, 'HADM_ID')) 
    #print("Init", df_new) 
    
    # remove duplicates
    df_rep = df_new.loc[:,df_new.columns.duplicated()]
    df_new = df_new.loc[:,~df_new.columns.duplicated()]   
    df_copy = df_new
    
    # add duplicates as new rowshttp://localhost:8888/notebooks/uio-research/EMBED_BIOPORTAL_DATA.ipynb
    #print("new", df_new)
    #print("rep", df_rep)
    #print("copy", df_copy)
    
    for name, value in df_rep.iteritems():
        df_add = df_copy.copy()
        df_add[name] = value.values[0]
        df_new = df_new.append(df_add, ignore_index=True)
    
    #print("PROD", df_new)
    if df.empty:
        df = df_new
    else:
        df = df.append(df_new, ignore_index=True)
    #print("--- Done ---")

In [ ]:
df_diag.set_index('HADM_ID', inplace=True).unstack('HADM_ID')

df_diag

In [ ]:
df2 = df_diag.pivot_table(columns='LABEL', values='ICD9_CODE', index='HADM_ID', aggfunc='any')

df2

In [ ]:
df_diag.set_index('HADM_ID', inplace=True)

In [ ]:
df_diag = df_diag[['ICD9_CODE', 'LABEL', 'HADM_ID']]

df_diag

In [ ]:
from itertools import permutations 

In [ ]:
n = "hallo ballo haha"
n.first()

In [ ]:
df_diag.pivot_table(columns='LABEL', values='ICD9_CODE', index=['HADM_ID', 'SEQ_NUM'], aggfunc='first')

In [ ]:
pd.DataFrame(df_diag.set_index('HADM_ID').unstack('LABEL'))

In [ ]:
df.to_csv('mimic-iii-clinical-database-1.4/DIAG_ROWS.csv', index=False)

In [ ]:
df = pd.read_csv('mimic-iii-clinical-database-1.4/DIAG_ROWS.csv')

df

In [ ]:
# fill in nan
df = df.dropna()

df

In [ ]:
df_loc = pd.read_csv('mimic-iii-clinical-database-1.4/ICUSTAYS.csv.gz')

df_loc = df_loc[['HADM_ID', 'ICUSTAY_ID']]

In [ ]:
df = pd.merge(df_loc, df, on='HADM_ID', how='left')

df

In [ ]:
df = df.drop(['HADM_ID'], axis=1)

df

In [ ]:
df = df.dropna()

df